In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np
import os
import astropy.units as u
import matplotlib.pyplot as plt

import snapshot_obj
import dataset_compute

import importlib

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(dataset_compute)

In [ ]:
# Get LCDM data:
sim_id = "V1_MR_fix"
snap_id = 127
LCDM = snapshot_obj.Snapshot(sim_id, snap_id)

# central:
gn = 2

In [ ]:
gns = LCDM.get_subhalos("GroupNumber")
sgns = LCDM.get_subhalos("SubGroupNumber")
sm = LCDM.get_subhalos("Stars/Mass") * u.g.to(u.Msun)
cops = LCDM.get_subhalos("CentreOfPotential") * u.cm.to(u.kpc)
mass = LCDM.get_subhalos('Mass') * u.g.to(u.Msun)

gal_centre = cops[np.logical_and(gns == gn, sgns == 0)]

# Choose galaxy satellites and with total mass sufficient for galaxy formation:
mask = np.logical_and.reduce((gns == gn, sgns != 0, mass > 10**8))
cops = cops[mask]
sm = sm[mask]

# Calculate distances to centre:
r = dataset_compute.periodic_wrap(LCDM, gal_centre, cops)
r = np.linalg.norm(cops - gal_centre, axis=1)
r_lum = r[sm > 0]
r_dark = r[sm == 0]

r_lum.sort()
r_dark.sort()

In [ ]:
fig, axes = plt.subplots()

axes.plot(r_lum, np.arange(1,r_lum.size+1), c="black", label='luminous')
axes.plot(r_dark, np.arange(1,r_dark.size+1), c="grey", label='dark')

axes.legend(loc=0)
axes.set_xlabel('$r[\mathrm{kpc}]$')
axes.set_ylabel('$N(<r)/N_\mathrm{tot}$')
axes.set_title('Satellites of halo gn={} by distance'.format(gn))

# Save figure:

filename = 'satellites_radial_dist_gn{}_{}.png'.format(gn,sim_id)

home = os.path.dirname(snapshot_obj.__file__)
path = os.path.join(home,"Figures")

## If the directory does not exist, create it
if not os.path.exists(path):
    os.makedirs(path)
plt.savefig(os.path.join(path,filename), dpi=200)